<a href="https://colab.research.google.com/github/jialusui/Lab-testing/blob/main/Analysis_step6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
import glob
import numpy as np
import timeit
import mapply

In [22]:
pip install mapply

     |████████████████████████████████| 81 kB 7.7 MB/s 
     |████████████████████████████████| 65 kB 3.9 MB/s 


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
%cd /content/drive/My Drive/Intern/

/content/drive/My Drive/Intern


In [14]:
ls

'Analysis step6.ipynb'        '~$output7.xlsx'
'handle date and time.ipynb'   output7.xlsx
 output1.xlsx                  output_null.xlsx
 output2.xlsx                  output_original.csv
 output3.xlsx                 '~$output.xlsx'
 output4.csv                   output.xlsx
 output5.xlsx                  Participants_data_for_Jialu_20211202/
 output6.csv                   public_all1.csv
 output7.csv                   Untitled.ipynb


In [15]:
public_all = pd.read_csv('public_all1.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
public_all

,Unnamed: 0,id,participant_unique_code,food_name,food_type,original_logtime,logtime,location_name,food_image_file_s3_location,original_logtime_notz,day,local_time,time,week_from_start,daypassed
0,0,640088,alqt16126987,Water,w,2017-03-02 04:19:00+00,2017-03-02 12:19:00+00,"Fielding Circle,Los Angeles, CA 90094",alqt16126987_w_2017-03-02_04-18-46.jpg,2017-03-02 04:19:00,2017-03-02,4.316667,04:19:00,10,68
1,1,640087,alqt16126987,Fish Oil Magnesium K2,m,2017-03-02 04:18:00+00,2017-03-02 12:18:00+00,"Fielding Circle,Los Angeles, CA 90094",alqt16126987_m_2017-03-02_04-18-16.jpg,2017-03-02 04:18:00,2017-03-02,4.300000,04:18:00,10,68
2,2,640086,alqt17028987,Coffee,b,2017-03-02 07:18:00+00,2017-03-02 12:18:00+00,"3195 Fireside Drive,Middleburg, FL 32068",alqt17028987_b_2017-03-02_07-18-07.jpg,2017-03-02 07:18:00,2017-03-02,7.300000,07:18:00,2,13
3,3,640085,alqt17029118,"Potatoes, Dumplings, Sausage",f,2017-03-02 13:17:00+00,2017-03-02 13:17:00+00,"11537, Všeradice, Česko",alqt17029118_f_2017-03-02_13-16-47.jpg,2017-03-02 13:17:00,2017-03-02,13.283333,13:17:00,2,8
4,4,640084,alqt17029226,water,w,2017-03-02 12:16:00+00,2017-03-02 12:16:00+00,"125 Wood St, London EC2V, UK",NaN,2017-03-02 12:16:00,2017-03-02,12.266667,12:16:00,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7592168,7592168,5389912,alqt1909100568,Breakfast Burrito,f,2019-10-02 19:43:00+00,2019-10-02 23:43:00+00,NaN,alqt1909100568_f_2019-10-02_19-44-59.jpg,2019-10-02 19:43:00,2019-10-02,19.716667,19:43:00,2,10
7592169,7592169,5389909,alqt180123900,Dinner,f,2019-10-02 19:44:00+00,2019-10-02 23:44:00+00,NaN,NaN,2019-10-02 19:44:00,2019-10-02,19.733333,19:44:00,92,637
7592170,7592170,5389907,alqt1909100872,Cashew,f,2019-10-02 18:33:00+00,2019-10-02 23:33:00+00,NaN,NaN,2019-10-02 18:33:00,2019-10-02,18.550000,18:33:00,1,3
7592171,7592171,5389906,alqt1909100883,6 Oat Cookies,f,2019-10-02 19:43:00+00,2019-10-02 23:43:00+00,NaN,NaN,2019-10-02 19:43:00,2019-10-02,19.716667,19:43:00,1,2


Note: Participants who logged at least two f/b per day with >5h difference between the first and the last f/b of the day will be considered compliant for food/beverage logging for that day.  E.g. (i) someone who logs a f/b at 6am and another one at 9am and nothing else for the day (until 3:59:59am of next calendar day) will be counted as non-compliant for that day. (ii) Someone who logs one f/b at 2am of a calendar day and another at 2pm of the same calendar day and nothing else, will be non-compliant because the 2am log will be counted towards the previous day. (iii) someone who logs at 9am and another f/b at2am of the next calendar day will be considered compliant because the day for our calculation goes from 4am till 3:59:59 am of the next calendar day. 


Notes: Since participants who logged only one f/b will not have any compliant day, exclude all participants from this calculation who logged only one f/b log in the app. This will exclude ~30% of participants.  


In [17]:
df_output = pd.read_csv('output7.csv')

In [18]:
df_output

,Unnamed: 0,participant_unique_code,total number of food_name,total number of food (f),total number of beverages (b),total number of water (w),total number of medication (m),the number of days,number of food name in week1&2,number of log days in week1&2,number of food name in week3,number of log days in week3,number of food name in week4,number of log days in week4,number of food name in week5,number of log days in week5,number of food name in week6,number of log days in week6,number of food name in week7,number of log days in week7,number of food name in week8,number of log days in week8,number of food name in week9,number of log days in week9,number of food name in week10,number of log days in week10,number of food name in week11,number of log days in week11,number of food name in week12,number of log days in week12,number of food name in week13,number of log days in week13,number of food name in week14,number of log days in week14,total number of food (f) in week1&2,total number of beverages (b) in week1&2,total number of water (w) in week1&2,total number of medication (m) in week1&2
0,0,alqt16126987,1224,698,146,283,97,77,153,14,42,7,67,7,56,7,66,6,46,7,46,7,32,6,33,7,25,5,23,6,29,7,29,7,88.0,18.0,33.0,14.0
1,1,alqt17028987,157,53,11,80,13,12,22,7,4,2,4,2,13,5,0,0,9,3,0,0,21,6,0,0,2,2,0,0,0,0,0,0,7.0,2.0,12.0,1.0
2,2,alqt17029118,98,79,9,9,1,25,27,14,6,4,3,2,6,2,3,3,3,2,1,1,2,2,3,3,4,2,1,1,4,4,1,1,25.0,2.0,0.0,0.0
3,3,alqt17029226,1213,554,118,498,43,93,572,14,147,7,70,7,58,7,59,7,35,7,29,6,47,7,33,6,25,7,12,3,24,6,10,6,263.0,55.0,231.0,23.0
4,4,alqt17029060,73,39,25,8,1,11,11,3,8,4,9,4,1,1,2,1,1,1,1,1,1,1,2,2,0,0,4,2,0,0,0,0,6.0,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77170,77170,alqt1910101064,8,5,1,2,0,1,8,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.0,1.0,2.0,0.0
77171,77171,alqt1910101109,11,4,1,6,0,1,11,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0,1.0,6.0,0.0
77172,77172,alqt1910101148,2,0,0,1,1,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.0,1.0
77173,77173,alqt1910101161,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0


In [19]:
df_output['total number of f and b'] = df_output['total number of food (f)'] + df_output['total number of beverages (b)']

In [20]:
df_output

,Unnamed: 0,participant_unique_code,total number of food_name,total number of food (f),total number of beverages (b),total number of water (w),total number of medication (m),the number of days,number of food name in week1&2,number of log days in week1&2,number of food name in week3,number of log days in week3,number of food name in week4,number of log days in week4,number of food name in week5,number of log days in week5,number of food name in week6,number of log days in week6,number of food name in week7,number of log days in week7,number of food name in week8,number of log days in week8,number of food name in week9,number of log days in week9,number of food name in week10,number of log days in week10,number of food name in week11,number of log days in week11,number of food name in week12,number of log days in week12,number of food name in week13,number of log days in week13,number of food name in week14,number of log days in week14,total number of food (f) in week1&2,total number of beverages (b) in week1&2,total number of water (w) in week1&2,total number of medication (m) in week1&2,total number of f and b
0,0,alqt16126987,1224,698,146,283,97,77,153,14,42,7,67,7,56,7,66,6,46,7,46,7,32,6,33,7,25,5,23,6,29,7,29,7,88.0,18.0,33.0,14.0,844
1,1,alqt17028987,157,53,11,80,13,12,22,7,4,2,4,2,13,5,0,0,9,3,0,0,21,6,0,0,2,2,0,0,0,0,0,0,7.0,2.0,12.0,1.0,64
2,2,alqt17029118,98,79,9,9,1,25,27,14,6,4,3,2,6,2,3,3,3,2,1,1,2,2,3,3,4,2,1,1,4,4,1,1,25.0,2.0,0.0,0.0,88
3,3,alqt17029226,1213,554,118,498,43,93,572,14,147,7,70,7,58,7,59,7,35,7,29,6,47,7,33,6,25,7,12,3,24,6,10,6,263.0,55.0,231.0,23.0,672
4,4,alqt17029060,73,39,25,8,1,11,11,3,8,4,9,4,1,1,2,1,1,1,1,1,1,1,2,2,0,0,4,2,0,0,0,0,6.0,2.0,2.0,1.0,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77170,77170,alqt1910101064,8,5,1,2,0,1,8,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.0,1.0,2.0,0.0,6
77171,77171,alqt1910101109,11,4,1,6,0,1,11,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0,1.0,6.0,0.0,5
77172,77172,alqt1910101148,2,0,0,1,1,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.0,1.0,0
77173,77173,alqt1910101161,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,1


In [31]:
#exclude participant who only have one foodlog:
include_list = list(df_output.loc[df_output['total number of f and b'] > 1].participant_unique_code)
include_index =  list(df_output.loc[df_output['total number of f and b'] > 1].index)

In [32]:
#exclude participant who only have one foodlog:
exclude_list = list(df_output.loc[df_output['total number of f and b'] <= 1].participant_unique_code)
exclude_index =  list(df_output.loc[df_output['total number of f and b'] <= 1].index)

In [26]:
print(len(include_list),'individual should be kept')
percent =len(exclude_list)/len(df_output)*100
print(len(exclude_list),f'individual should be excluded, which is {percent} percent')

61675 individual should be kept
15500 individual should be excluded, which is 20.084224165856817 percent


In [27]:
#exclude data: some participants entered some water/medications besides 1 f/b
public_all.loc[public_all.participant_unique_code.isin(exclude_list)]

,Unnamed: 0,id,participant_unique_code,food_name,food_type,original_logtime,logtime,location_name,food_image_file_s3_location,original_logtime_notz,day,local_time,time,week_from_start,daypassed
52,52,640036,alqt17039324,water,w,2017-03-02 06:38:00+00,2017-03-02 11:38:00+00,"3436 E Main St, College Park, GA 30337, USA",alqt17039324_w_2017-03-02_06-38-04.jpg,2017-03-02 06:38:00,2017-03-02,6.633333,06:38:00,1,0
236,236,639847,alqt17039314,Coffe,b,2017-03-02 07:51:00+00,2017-03-02 07:51:00+00,NaN,alqt17039314_b_2017-03-02_07-51-22.jpg,2017-03-02 07:51:00,2017-03-02,7.850000,07:51:00,1,0
277,277,639804,alqt17039314,Water,w,2017-03-02 06:52:00+00,2017-03-02 06:52:00+00,NaN,NaN,2017-03-02 06:52:00,2017-03-02,6.866667,06:52:00,1,0
807,807,639259,alqt17029248,Cheese,f,2017-03-02 00:42:00+00,2017-03-02 00:42:00+00,"118 Rutledge Ave, Charleston, SC 29401, USA",NaN,2017-03-02 00:42:00,2017-03-01,24.700000,00:42:00,1,0
1418,1418,638647,alqt17039307,Turkey Stew,f,2017-03-01 11:48:00+00,2017-03-01 19:48:00+00,NaN,alqt17039307_f_2017-03-01_11-47-29.jpg,2017-03-01 11:48:00,2017-03-01,11.800000,11:48:00,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7590700,7590700,5391771,alqt1910101148,"Tea, Apple Cider Vinegar Tea",m,2019-10-03 05:30:00+00,2019-10-03 12:30:00+00,NaN,NaN,2019-10-03 05:30:00,2019-10-03,5.500000,05:30:00,1,0
7590718,7590718,5391751,alqt1910101161,Juice,f,2019-10-03 08:30:00+00,2019-10-03 14:30:00+00,NaN,alqt1910101161_f_2019-10-03_08-30-00.jpg,2019-10-03 08:30:00,2019-10-03,8.500000,08:30:00,1,0
7590721,7590721,5391748,alqt1910101148,Water,w,2019-10-03 07:19:00+00,2019-10-03 14:19:00+00,NaN,NaN,2019-10-03 07:19:00,2019-10-03,7.316667,07:19:00,1,0
7591847,7591847,5390349,alqt1910101070,Poha,f,2019-10-03 07:46:00+00,2019-10-03 02:16:00+00,NaN,NaN,2019-10-03 07:46:00,2019-10-03,7.766667,07:46:00,1,0


In [28]:

filter1 = public_all.loc[public_all.participant_unique_code.isin(include_list)]


Note: Participants who logged at least two f/b per day with >5h difference between the first and the last f/b of the day will be considered compliant for food/beverage logging for that day. E.g. (i) someone who logs a f/b at 6am and another one at 9am and nothing else for the day (until 3:59:59am of next calendar day) will be counted as non-compliant for that day. (ii) Someone who logs one f/b at 2am of a calendar day and another at 2pm of the same calendar day and nothing else, will be non-compliant because the 2am log will be counted towards the previous day. (iii) someone who logs at 9am and another f/b at2am of the next calendar day will be considered compliant because the day for our calculation goes from 4am till 3:59:59 am of the next calendar day.

In [29]:
filter1

,Unnamed: 0,id,participant_unique_code,food_name,food_type,original_logtime,logtime,location_name,food_image_file_s3_location,original_logtime_notz,day,local_time,time,week_from_start,daypassed
0,0,640088,alqt16126987,Water,w,2017-03-02 04:19:00+00,2017-03-02 12:19:00+00,"Fielding Circle,Los Angeles, CA 90094",alqt16126987_w_2017-03-02_04-18-46.jpg,2017-03-02 04:19:00,2017-03-02,4.316667,04:19:00,10,68
1,1,640087,alqt16126987,Fish Oil Magnesium K2,m,2017-03-02 04:18:00+00,2017-03-02 12:18:00+00,"Fielding Circle,Los Angeles, CA 90094",alqt16126987_m_2017-03-02_04-18-16.jpg,2017-03-02 04:18:00,2017-03-02,4.300000,04:18:00,10,68
2,2,640086,alqt17028987,Coffee,b,2017-03-02 07:18:00+00,2017-03-02 12:18:00+00,"3195 Fireside Drive,Middleburg, FL 32068",alqt17028987_b_2017-03-02_07-18-07.jpg,2017-03-02 07:18:00,2017-03-02,7.300000,07:18:00,2,13
3,3,640085,alqt17029118,"Potatoes, Dumplings, Sausage",f,2017-03-02 13:17:00+00,2017-03-02 13:17:00+00,"11537, Všeradice, Česko",alqt17029118_f_2017-03-02_13-16-47.jpg,2017-03-02 13:17:00,2017-03-02,13.283333,13:17:00,2,8
4,4,640084,alqt17029226,water,w,2017-03-02 12:16:00+00,2017-03-02 12:16:00+00,"125 Wood St, London EC2V, UK",NaN,2017-03-02 12:16:00,2017-03-02,12.266667,12:16:00,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7592168,7592168,5389912,alqt1909100568,Breakfast Burrito,f,2019-10-02 19:43:00+00,2019-10-02 23:43:00+00,NaN,alqt1909100568_f_2019-10-02_19-44-59.jpg,2019-10-02 19:43:00,2019-10-02,19.716667,19:43:00,2,10
7592169,7592169,5389909,alqt180123900,Dinner,f,2019-10-02 19:44:00+00,2019-10-02 23:44:00+00,NaN,NaN,2019-10-02 19:44:00,2019-10-02,19.733333,19:44:00,92,637
7592170,7592170,5389907,alqt1909100872,Cashew,f,2019-10-02 18:33:00+00,2019-10-02 23:33:00+00,NaN,NaN,2019-10-02 18:33:00,2019-10-02,18.550000,18:33:00,1,3
7592171,7592171,5389906,alqt1909100883,6 Oat Cookies,f,2019-10-02 19:43:00+00,2019-10-02 23:43:00+00,NaN,NaN,2019-10-02 19:43:00,2019-10-02,19.716667,19:43:00,1,2


In [33]:
public_all.iloc[exclude_index]['food_type'].unique()

array(['b', 'f', 'w', 'm'], dtype=object)

In [34]:
def check_compliance(s):
    #s: dataframe of a single day for a specific participant
    s=s.sort_values(by=['local_time'])
    first_logtime = s.iloc[0].local_time
    last_logtime = s.iloc[-1].local_time
    #print(first_logtime)
    #print(last_logtime)
    if last_logtime-first_logtime >5:
        return True
    else:
        return False
    

In [35]:
test1 = filter1.query('participant_unique_code=="alqt17029118" and daypassed == 11 and (food_type == "f" or food_type == "b")')

In [36]:
test1

,Unnamed: 0,id,participant_unique_code,food_name,food_type,original_logtime,logtime,location_name,food_image_file_s3_location,original_logtime_notz,day,local_time,time,week_from_start,daypassed
1585763,1585763,646552,alqt17029118,Vegetables With Rice,f,2017-03-05 15:42:00+00,2017-03-05 15:42:00+00,"Na Dolinách 275/9, Podolí, 147 00 Praha-Praha ...",alqt17029118_f_2017-03-05_15-42-35.jpg,2017-03-05 15:42:00,2017-03-05,15.700000,15:42:00,2,11
1585862,1585862,646453,alqt17029118,Soup,f,2017-03-05 14:31:00+00,2017-03-05 14:31:00+00,"Švehlova 1391/32, 102 00 Praha 10-Hostivař, Česko",alqt17029118_f_2017-03-05_14-31-49.jpg,2017-03-05 14:31:00,2017-03-05,14.516667,14:31:00,2,11
1585911,1585911,646404,alqt17029118,Orange,f,2017-03-05 13:56:00+00,2017-03-05 13:56:00+00,"Mattioliho 18, 3250, 106 00 Praha-Praha 10, Česko",alqt17029118_f_2017-03-05_13-56-14.jpg,2017-03-05 13:56:00,2017-03-05,13.933333,13:56:00,2,11


In [37]:
check_compliance(test1)

False

Find the compliant days for each day of the first 14 weeks (98 days) for each participant.  For each participant, day 1 is the date on which they first logged any food item in the app.

The above steps should produce a table or file containing each participant ID followed by days 1-98 (0-97), whether they were compliant or not (Y/N) for each of those days, no. of food_name (total logs), f/b logs for each day, no. of m logs and no. of w log for each day. 

Find the number of compliant days for the first 14 days, and do the same for each of the subsequent weeks till week 14. This will be the most intensive analyses so far. It may take a few hours of computing time. 
 
Find participants who had at least 10 compliant days for the first 14 days of data logging. You may find ~20,000 participants meeting this criteria. 
In subsequent analyses we will use the data from participants who had at least 10 compliant days in the first 14 days. 


In [38]:
df_compliance = pd.DataFrame(data = filter1.participant_unique_code.unique(),columns=['unique_code'])

In [39]:
filter1.query('participant_unique_code =="alqt17029226"')

,Unnamed: 0,id,participant_unique_code,food_name,food_type,original_logtime,logtime,location_name,food_image_file_s3_location,original_logtime_notz,day,local_time,time,week_from_start,daypassed
4,4,640084,alqt17029226,water,w,2017-03-02 12:16:00+00,2017-03-02 12:16:00+00,"125 Wood St, London EC2V, UK",NaN,2017-03-02 12:16:00,2017-03-02,12.266667,12:16:00,1,4
27,27,640061,alqt17029226,water,w,2017-03-02 11:57:00+00,2017-03-02 11:57:00+00,"125 Wood St, London EC2V, UK",NaN,2017-03-02 11:57:00,2017-03-02,11.950000,11:57:00,1,4
65,65,640023,alqt17029226,water,w,2017-03-02 11:16:00+00,2017-03-02 11:16:00+00,"125 Wood St, London EC2V, UK",NaN,2017-03-02 11:16:00,2017-03-02,11.266667,11:16:00,1,4
111,111,639973,alqt17029226,water,w,2017-03-02 10:24:00+00,2017-03-02 10:24:00+00,"125 Wood St, London EC2V, UK",NaN,2017-03-02 10:24:00,2017-03-02,10.400000,10:24:00,1,4
171,171,639913,alqt17029226,Decaf Coffee,b,2017-03-02 09:12:00+00,2017-03-02 09:12:00+00,"125 Wood St, London EC2V, UK",NaN,2017-03-02 09:12:00,2017-03-02,9.200000,09:12:00,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2165276,2165276,2293527,alqt17029226,Vegan Stew,f,2018-06-21 13:32:00+00,2018-06-21 12:32:00+00,"125 Wood St, London EC2V 7AN, UK",NaN,2018-06-21 13:32:00,2018-06-21,13.533333,13:32:00,69,480
2165895,2165895,2292762,alqt17029226,Banana Bread,f,2018-06-21 11:28:00+00,2018-06-21 10:28:00+00,"125 Wood St, London EC2V 7AN, UK",alqt17029226_f_2018-06-21_11-28-35.jpg,2018-06-21 11:28:00,2018-06-21,11.466667,11:28:00,69,480
2166134,2166134,2292430,alqt17029226,water,w,2018-06-21 10:13:00+00,2018-06-21 09:13:00+00,"125 Wood St, London EC2V 7AN, UK",NaN,2018-06-21 10:13:00,2018-06-21,10.216667,10:13:00,69,480
2166389,2166389,2292128,alqt17029226,"Yoghurt, Berries, Homemade Granola, Maple Syrup",f,2018-06-21 08:52:00+00,2018-06-21 07:52:00+00,"125 Wood St, London EC2V 7AN, UK",alqt17029226_f_2018-06-21_08-53-47.jpg,2018-06-21 08:52:00,2018-06-21,8.866667,08:52:00,69,480


In [46]:
def find_compliance(s,df):
    par = s.unique_code
    df_test = df.query('participant_unique_code == @par')
    if len(df_test)<2:
        return False
    return check_compliance(df_test)

In [41]:
 df_fb = filter1.query('food_type == "f" or food_type == "b"')

In [42]:
df_fb

,Unnamed: 0,id,participant_unique_code,food_name,food_type,original_logtime,logtime,location_name,food_image_file_s3_location,original_logtime_notz,day,local_time,time,week_from_start,daypassed
2,2,640086,alqt17028987,Coffee,b,2017-03-02 07:18:00+00,2017-03-02 12:18:00+00,"3195 Fireside Drive,Middleburg, FL 32068",alqt17028987_b_2017-03-02_07-18-07.jpg,2017-03-02 07:18:00,2017-03-02,7.300000,07:18:00,2,13
3,3,640085,alqt17029118,"Potatoes, Dumplings, Sausage",f,2017-03-02 13:17:00+00,2017-03-02 13:17:00+00,"11537, Všeradice, Česko",alqt17029118_f_2017-03-02_13-16-47.jpg,2017-03-02 13:17:00,2017-03-02,13.283333,13:17:00,2,8
5,5,640083,alqt17029060,1 Candy,f,2017-03-02 11:30:00+00,2017-03-02 10:30:00+00,"Via San Barnaba, 18,20122 Milano",NaN,2017-03-02 11:30:00,2017-03-02,11.500000,11:30:00,2,10
6,6,640082,alqt17028793,Coffee W Cream,b,2017-03-02 07:13:00+00,2017-03-02 12:13:00+00,"120 Kentucky Ave SE, Washington, DC 20003, USA",alqt17028793_b_2017-03-02_07-09-51.jpg,2017-03-02 07:13:00,2017-03-02,7.216667,07:13:00,3,20
9,9,640079,alqt17029167,Coffee With Milk,b,2017-03-02 13:12:00+00,2017-03-02 12:12:00+00,"Am Klopferspitz 18,82152 Planegg",NaN,2017-03-02 13:12:00,2017-03-02,13.200000,13:12:00,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7592168,7592168,5389912,alqt1909100568,Breakfast Burrito,f,2019-10-02 19:43:00+00,2019-10-02 23:43:00+00,NaN,alqt1909100568_f_2019-10-02_19-44-59.jpg,2019-10-02 19:43:00,2019-10-02,19.716667,19:43:00,2,10
7592169,7592169,5389909,alqt180123900,Dinner,f,2019-10-02 19:44:00+00,2019-10-02 23:44:00+00,NaN,NaN,2019-10-02 19:44:00,2019-10-02,19.733333,19:44:00,92,637
7592170,7592170,5389907,alqt1909100872,Cashew,f,2019-10-02 18:33:00+00,2019-10-02 23:33:00+00,NaN,NaN,2019-10-02 18:33:00,2019-10-02,18.550000,18:33:00,1,3
7592171,7592171,5389906,alqt1909100883,6 Oat Cookies,f,2019-10-02 19:43:00+00,2019-10-02 23:43:00+00,NaN,NaN,2019-10-02 19:43:00,2019-10-02,19.716667,19:43:00,1,2


In [43]:
%%timeit
df_fb.query('participant_unique_code == "alqt17029226" and daypassed == 0')

10 loops, best of 5: 139 ms per loop


In [44]:
mapply.init(
    n_workers=-1,
    chunk_size=100,
    max_chunks_per_worker=8,
    progressbar=False
)

In [48]:
df_0 = df_fb.query('daypassed==0')

In [53]:

df_test = df_fb.query('participant_unique_code == "alqt17029118" and daypassed==0')

In [55]:
%%timeit
find_compliance(df_compliance.iloc[0],df_test)

100 loops, best of 5: 2.45 ms per loop


In [61]:
list(range(1,11))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

we need to calculate day 0 to day 97, so group it into 10:

In [64]:
from tqdm.notebook import tqdm

In [56]:
df_0 = df_fb.query('daypassed==0')
df_compliance['compliant days for day 1'] = df_compliance.mapply(lambda x: (find_compliance(x, df_0)), axis=1)

In [63]:
df_compliance

,unique_code,compliant days for day 1,compliant days for day 2,compliant days for day 3,compliant days for day 4
0,alqt16126987,True,True,True,True
1,alqt17028987,True,True,False,True
2,alqt17029118,True,True,True,False
3,alqt17029226,True,True,True,True
4,alqt17029060,True,True,True,True
...,...,...,...,...,...
61670,alqt1910101146,False,False,False,False
61671,alqt1910101160,False,False,False,False
61672,alqt1910101197,False,False,False,False
61673,alqt1910101064,False,False,False,False


In [ ]:
#day 1 - 10
for day in range(1,11):
  df = df_fb.query('daypassed==@day')
  df_compliance[f'compliant days for day {day}'] = df_compliance.mapply(lambda x: (find_compliance(x, df)), axis=1)

In [65]:
#day 1 - 10
for day in tqdm(range(4,7)):
  df = df_fb.query('daypassed==@day')
  df_compliance[f'compliant days for day {day}'] = df_compliance.mapply(lambda x: (find_compliance(x, df)), axis=1)

  0%|          | 0/3 [00:00<?, ?it/s]

In [67]:
df_compliance.isna().sum()

unique_code                 0
compliant days for day 1    0
compliant days for day 2    0
compliant days for day 3    0
compliant days for day 4    0
compliant days for day 5    0
compliant days for day 6    0
dtype: int64

In [68]:
df_compliance.to_csv('compliant.csv')

In [ ]:
#day 11 - 20
for day in tqdm(range(11,21)):
  df = df_fb.query('daypassed==@day')
  df_compliance[f'compliant days for day {day}'] = df_compliance.mapply(lambda x: (find_compliance(x, df)), axis=1)

In [ ]:
#day 21 - 30
for day in tqdm(range(21,31)):
  df = df_fb.query('daypassed==@day')
  df_compliance[f'compliant days for day {day}'] = df_compliance.mapply(lambda x: (find_compliance(x, df)), axis=1)

In [ ]:
#day 31 - 40
for day in tqdm(range(31,41)):
  df = df_fb.query('daypassed==@day')
  df_compliance[f'compliant days for day {day}'] = df_compliance.mapply(lambda x: (find_compliance(x, df)), axis=1)

In [ ]:
#day 41 - 50
for day in tqdm(range(41,51)):
  df = df_fb.query('daypassed==@day')
  df_compliance[f'compliant days for day {day}'] = df_compliance.mapply(lambda x: (find_compliance(x, df)), axis=1)

In [ ]:
#day 51 - 61
for day in tqdm(range(51,61)):
  df = df_fb.query('daypassed==@day')
  df_compliance[f'compliant days for day {day}'] = df_compliance.mapply(lambda x: (find_compliance(x, df)), axis=1)

In [ ]:
#day 61 - 70
for day in tqdm(range(61,71)):
  df = df_fb.query('daypassed==@day')
  df_compliance[f'compliant days for day {day}'] = df_compliance.mapply(lambda x: (find_compliance(x, df)), axis=1)

In [ ]:
#day 71 - 80
for day in tqdm(range(71,81)):
  df = df_fb.query('daypassed==@day')
  df_compliance[f'compliant days for day {day}'] = df_compliance.mapply(lambda x: (find_compliance(x, df)), axis=1)

In [ ]:
#day 81 - 90
for day in tqdm(range(81,91)):
  df = df_fb.query('daypassed==@day')
  df_compliance[f'compliant days for day {day}'] = df_compliance.mapply(lambda x: (find_compliance(x, df)), axis=1)

In [ ]:
#day 91 - 97
for day in tqdm(range(91,98)):
  df = df_fb.query('daypassed==@day')
  df_compliance[f'compliant days for day {day}'] = df_compliance.mapply(lambda x: (find_compliance(x, df)), axis=1)

In [57]:
df_compliance

,unique_code,compliant days for day 1
0,alqt16126987,True
1,alqt17028987,True
2,alqt17029118,False
3,alqt17029226,False
4,alqt17029060,True
...,...,...
61670,alqt1910101146,False
61671,alqt1910101160,False
61672,alqt1910101197,False
61673,alqt1910101064,True


In [ ]:
#https://pypi.org/project/mapply/